In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plot
import matplotlib
import pickle

In [ ]:
plot_every_loc= True # True = plot also every single location, False = only overall plot for all locations (loop locations anyways to extract data)

well_list = pd.read_csv("./locations.csv",sep=';',header = 0,encoding = 'Latin1')

GWL_scaled_list = []
winter_contrib_sum_P_list = []
winter_contrib_sum_T_list = []
spring_contrib_sum_P_list = []
spring_contrib_sum_T_list = []
summer_contrib_sum_P_list = []
summer_contrib_sum_T_list = []
fall_contrib_sum_P_list = []
fall_contrib_sum_T_list = []

for loc in range(well_list.shape[0]):
   
    ID = well_list.alias[loc]
    print(ID+": "+str(loc))
    
    method = 'lrp.z'
    file = './Results_XAI/analysis_'+ID+'_'+method.replace('.','')+'.pickle'
    
    #get the data for loc
    with open(file, 'rb') as f:
        imports = pickle.load(f)
    
    analysis = imports['analysis_mean']
    data_investigate = imports['data_investigate']
    data_investigate_ext = imports['data_investigate_ext']
    
    seqlength = 52
    y_list = imports['simulations']
    sim_members = np.zeros(shape = (y_list[0].shape[0],20))
    for i in range(20):
        sim_members[:,i] = y_list[i].reshape(-1,)
    sim_mean = np.nanmean(sim_members,axis = 1)
    sim_uncertainty = [np.quantile(sim_members, 0.05, axis=1),np.quantile(sim_members, 0.95, axis=1)]
    
    
    #%% prepare data for plotting
    T_reshaped = np.zeros(shape=(data_investigate.shape[0],seqlength))
    T_reshaped[:]=np.nan
    P_reshaped = T_reshaped.copy()
    for i in range(T_reshaped.shape[0]):
        T_reshaped[i,:] = data_investigate_ext['T'][i:i+seqlength]
        P_reshaped[i,:] = data_investigate_ext['P'][i:i+seqlength]

    T_reshaped_flat = T_reshaped.reshape(-1,1)
    P_reshaped_flat = P_reshaped.reshape(-1,1)
    analysis_P_flat = analysis[:,:,0].reshape(-1,)
    analysis_T_flat = analysis[:,:,1].reshape(-1,)
    
    steps = np.linspace(seqlength,1,seqlength)
    steps = steps.reshape(1,-1)
    steps = np.repeat(steps,T_reshaped.shape[0],axis=0)
    steps_flat = steps.reshape(-1,1)
        
    analysis_reshaped2 = np.zeros(shape=(data_investigate.shape[0],data_investigate.shape[0]+seqlength,2))
    analysis_reshaped2[:]=np.nan
    T_reshaped2 = np.zeros(shape=(data_investigate.shape[0],data_investigate.shape[0]+seqlength))
    T_reshaped2[:]=np.nan
    P_reshaped2 = T_reshaped2.copy()
    
    for i in range(data_investigate.shape[0]):
        analysis_reshaped2[i,i:i+seqlength,0] = analysis[i,:,0]
        analysis_reshaped2[i,i:i+seqlength,1] = analysis[i,:,1]
        P_reshaped2[i,i:i+seqlength] = P_reshaped[i,:]
        T_reshaped2[i,i:i+seqlength] = T_reshaped[i,:]
    
    
    #define seasons
    mask_winter = ((data_investigate_ext.index.month == 12) | 
                    (data_investigate_ext.index.month == 1) |
                    (data_investigate_ext.index.month == 2))
    
    mask_spring = ((data_investigate_ext.index.month == 3) |
                    (data_investigate_ext.index.month == 4) | 
                    (data_investigate_ext.index.month == 5))
    
    mask_summer = ((data_investigate_ext.index.month == 6) |
                    (data_investigate_ext.index.month == 7) | 
                    (data_investigate_ext.index.month == 8))
    
    mask_fall = ((data_investigate_ext.index.month == 9) |
                    (data_investigate_ext.index.month == 10) | 
                    (data_investigate_ext.index.month == 11))
    
    mask_GWL_target_period = ((data_investigate.index.month == 7) |
                                (data_investigate.index.month == 8) | 
                                (data_investigate.index.month == 9))
    
    GWL_target_period = data_investigate['GWL'][mask_GWL_target_period]
    temp = analysis_reshaped2[mask_GWL_target_period,:,:]
    winter_contribution = temp[:,mask_winter,:]
    spring_contribution = temp[:,mask_spring,:]
    summer_contribution = temp[:,mask_summer,:]
    fall_contribution = temp[:,mask_fall,:]
    
    winter_contrib_sum_P = np.nansum(winter_contribution[:,:,0],axis=1)
    winter_contrib_sum_T = np.nansum(winter_contribution[:,:,1],axis=1)
    spring_contrib_sum_P = np.nansum(spring_contribution[:,:,0],axis=1)
    spring_contrib_sum_T = np.nansum(spring_contribution[:,:,1],axis=1)
    summer_contrib_sum_P = np.nansum(summer_contribution[:,:,0],axis=1)
    summer_contrib_sum_T = np.nansum(summer_contribution[:,:,1],axis=1)
    fall_contrib_sum_P = np.nansum(fall_contribution[:,:,0],axis=1)
    fall_contrib_sum_T = np.nansum(fall_contribution[:,:,1],axis=1)
    
    GWL_target_period = (GWL_target_period - GWL_target_period.min(axis=0)) / (GWL_target_period.max(axis=0) - GWL_target_period.min(axis=0))
    GWL_scaled_list.append(GWL_target_period)
    winter_contrib_sum_P_list.append(winter_contrib_sum_P)
    winter_contrib_sum_T_list.append(winter_contrib_sum_T)
    spring_contrib_sum_P_list.append(spring_contrib_sum_P)
    spring_contrib_sum_T_list.append(spring_contrib_sum_T)
    summer_contrib_sum_P_list.append(summer_contrib_sum_P)
    summer_contrib_sum_T_list.append(summer_contrib_sum_T)
    fall_contrib_sum_P_list.append(fall_contrib_sum_P)
    fall_contrib_sum_T_list.append(fall_contrib_sum_T)

    #plot individual location
    if plot_every_loc:
        cmap = matplotlib.cm.get_cmap("seismic_r")
        lb = np.min([np.nanmin(winter_contrib_sum_P),np.nanmin(winter_contrib_sum_T),
                        np.nanmin(spring_contrib_sum_P),np.nanmin(spring_contrib_sum_T),
                        np.nanmin(summer_contrib_sum_P),np.nanmin(summer_contrib_sum_T),
                        np.nanmin(fall_contrib_sum_P),np.nanmin(fall_contrib_sum_T),
                        ])-0.05
        ub = np.max([np.nanmax(winter_contrib_sum_P),np.nanmax(winter_contrib_sum_T),
                        np.nanmax(spring_contrib_sum_P),np.nanmax(spring_contrib_sum_T),
                        np.nanmax(summer_contrib_sum_P),np.nanmax(summer_contrib_sum_T),
                        np.nanmax(fall_contrib_sum_P),np.nanmax(fall_contrib_sum_T),
                        ])+0.05
        max_abs = np.max([np.abs(lb),np.abs(ub)])
        lb = -max_abs
        ub = max_abs

        fig, ax = plot.subplots(1,4,figsize=(12,3.5))

        for axs in ax:
            axs.axhline(0,color='darkgrey',linewidth=1,alpha=0.7)
            axs.axvline(0,color='darkgrey',linewidth=1,alpha=0.7)

        # Winter, P vs T
        ax[0].scatter(winter_contrib_sum_P,winter_contrib_sum_T,c=GWL_target_period,cmap=cmap,s=2)
        ax[0].set_xlabel('P')
        ax[0].set_ylabel('T')
        ax[0].set_title('Winter contributions')

        # spring, P vs T
        ax[1].scatter(spring_contrib_sum_P,spring_contrib_sum_T,c=GWL_target_period,cmap=cmap,s=2)
        ax[1].set_xlabel('P')
        ax[1].set_ylabel('T')
        ax[1].set_title('Spring contributions')
                
        # summer, P vs T
        ax[2].scatter(summer_contrib_sum_P,summer_contrib_sum_T,c=GWL_target_period,cmap=cmap,s=2)
        ax[2].set_xlabel('P')
        ax[2].set_ylabel('T')
        ax[2].set_title('Summer contributions')

        # fall, P vs T
        scat = ax[3].scatter(fall_contrib_sum_P,fall_contrib_sum_T,c=GWL_target_period,cmap=cmap,s=2)
        ax[3].set_xlabel('P')
        ax[3].set_ylabel('T')
        ax[3].set_title('Fall contributions')

        for axs in ax:
            axs.set_xlim([lb,ub])
            axs.set_ylim([lb,ub])
            axs.grid(alpha=0.2)
            
        cbar_ax = fig.add_axes([1.01, 0.25, 0.01, 0.5])
        cbar = fig.colorbar(scat, cax=cbar_ax,shrink=0.8,aspect=20)#,anchor =(-0.25,0.5))
        cbar.set_label(label="GWL [ ]",rotation=90)
        cbar.set_ticks([0,1])
        cbar.ax.set_yticklabels(['low','high'],rotation=90)

        plot.suptitle(ID+': influence of seasons on low-water periods')
        plot.tight_layout()
        plot.savefig('./Results_XAI/seasonal_contribution_scatter_plot/'+ID+'.png',bbox_inches='tight')
        plot.show()
        
        
     

In [ ]:
#prepare data for all locations plot
loc=0
GWL_target_period = GWL_scaled_list[loc]
winter_contrib_sum_P = winter_contrib_sum_P_list[loc]
winter_contrib_sum_T= winter_contrib_sum_T_list[loc]
spring_contrib_sum_P= spring_contrib_sum_P_list[loc]
spring_contrib_sum_T= spring_contrib_sum_T_list[loc]
summer_contrib_sum_P= summer_contrib_sum_P_list[loc]
summer_contrib_sum_T= summer_contrib_sum_T_list[loc]
fall_contrib_sum_P= fall_contrib_sum_P_list[loc]
fall_contrib_sum_T= fall_contrib_sum_T_list[loc]


for loc in range(1,well_list.shape[0]):
    GWL_target_period = np.concatenate((GWL_target_period,GWL_scaled_list[loc]),axis=0)
    winter_contrib_sum_P = np.concatenate((winter_contrib_sum_P,winter_contrib_sum_P_list[loc]),axis=0)
    winter_contrib_sum_T= np.concatenate((winter_contrib_sum_T, winter_contrib_sum_T_list[loc]),axis=0)
    spring_contrib_sum_P= np.concatenate((spring_contrib_sum_P, spring_contrib_sum_P_list[loc]),axis=0)
    spring_contrib_sum_T= np.concatenate((spring_contrib_sum_T, spring_contrib_sum_T_list[loc]),axis=0)
    summer_contrib_sum_P= np.concatenate((summer_contrib_sum_P, summer_contrib_sum_P_list[loc]),axis=0)
    summer_contrib_sum_T= np.concatenate((summer_contrib_sum_T, summer_contrib_sum_T_list[loc]),axis=0)
    fall_contrib_sum_P= np.concatenate((fall_contrib_sum_P, fall_contrib_sum_P_list[loc]),axis=0)
    fall_contrib_sum_T= np.concatenate((fall_contrib_sum_T, fall_contrib_sum_T_list[loc]),axis=0)

#control plotting colors
cmap = matplotlib.cm.get_cmap("seismic_r")
lb = np.min([np.nanmin(winter_contrib_sum_P),np.nanmin(winter_contrib_sum_T),#lower bound
             np.nanmin(spring_contrib_sum_P),np.nanmin(spring_contrib_sum_T),
             np.nanmin(summer_contrib_sum_P),np.nanmin(summer_contrib_sum_T),
             np.nanmin(fall_contrib_sum_P),np.nanmin(fall_contrib_sum_T),
             ])-0.05
ub = np.max([np.nanmax(winter_contrib_sum_P),np.nanmax(winter_contrib_sum_T),#upper bound
             np.nanmax(spring_contrib_sum_P),np.nanmax(spring_contrib_sum_T),
             np.nanmax(summer_contrib_sum_P),np.nanmax(summer_contrib_sum_T),
             np.nanmax(fall_contrib_sum_P),np.nanmax(fall_contrib_sum_T),
             ])+0.05
max_abs = np.max([np.abs(lb),np.abs(ub)])
lb = -max_abs
ub = max_abs

#make plot
fig, ax = plot.subplots(1,4,figsize=(12,3.5))

for axs in ax:
    axs.axhline(0,color='darkgrey',linewidth=1,alpha=0.7)
    axs.axvline(0,color='darkgrey',linewidth=1,alpha=0.7)

# Winter, P vs T
ax[0].scatter(winter_contrib_sum_P,winter_contrib_sum_T,c=GWL_target_period,cmap=cmap,s=2)
ax[0].set_xlabel('P')
ax[0].set_ylabel('T')
ax[0].set_title('Winter contributions')

# spring, P vs T
ax[1].scatter(spring_contrib_sum_P,spring_contrib_sum_T,c=GWL_target_period,cmap=cmap,s=2)
ax[1].set_xlabel('P')
ax[1].set_ylabel('T')
ax[1].set_title('Spring contributions')
        
# summer, P vs T
ax[2].scatter(summer_contrib_sum_P,summer_contrib_sum_T,c=GWL_target_period,cmap=cmap,s=2)
ax[2].set_xlabel('P')
ax[2].set_ylabel('T')
ax[2].set_title('Summer contributions')

# fall, P vs T
scat = ax[3].scatter(fall_contrib_sum_P,fall_contrib_sum_T,c=GWL_target_period,cmap=cmap,s=2)
ax[3].set_xlabel('P')
ax[3].set_ylabel('T')
ax[3].set_title('Fall contributions')

for axs in ax:
    axs.set_xlim([lb,ub])
    axs.set_ylim([lb,ub])
    axs.grid(alpha=0.2)
    
cbar_ax = fig.add_axes([1.01, 0.25, 0.01, 0.5])
cbar = fig.colorbar(scat, cax=cbar_ax,shrink=0.8,aspect=20)#,anchor =(-0.25,0.5))
cbar.set_label(label="GWL [ ]",rotation=90)
cbar.set_ticks([0,1])
cbar.ax.set_yticklabels(['low','high'],rotation=90)

plot.suptitle('Influence of seasons on low-water periods')
plot.tight_layout()
plot.savefig('./Results_XAI/seasonal_contribution_scatter_plot/all_locations.png',bbox_inches='tight')
plot.show()